In [1]:
# from math import log, sqrt, pi, exp
# from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import scipy
from scipy import sparse
from scipy.sparse import linalg
import pandas as pd
from pandas import DataFrame
from matplotlib import pyplot as plt
import copy
from collections import OrderedDict

from fd_adi import * 

In [2]:
T = 1
Nx = np.array([4,4]) # x and v spaces
Nx = np.array([2**5,2**5]) # x and v spaces
# Nx = np.array([2**6,2**6]) # x and v spaces
# Nx = np.array([2**10,2**6]) # x and v spaces
x_max = np.array([1.5,0.3]) # log(price), vol-vol
x_min = np.array([-2,0.0])
dx = (x_max-x_min)/(Nx-1)
Nt = 12 # always scalar
t_space = np.linspace(0,T,Nt+1)
dt = t_space[1]-t_space[0]

In [3]:
theta = 0.05; kappa = 0.3; sigma = 0.5; rho = -0.6; v0 = 0.04
r = 0.0 # always presume zero risk-free interest
S0 = 1.0; k = 1.0;
x0 = np.log(S0); log_k = np.log(k)

# Let's plan
# For a test, we can start with dense matrix. Don't complicate too much at the beginning.
# First, think about Pxv
# ux(x,y) = (u(x+1,y)-u(x,y))/dx
# uxy(x,y) = (u(x+1,y+1) + u(x-1,y-1) - u(x+1,y-1) - u(x-1,y+1))/(4dxdy) # 2dx,2dy
# uxy(x,y) = (u(x+1,y+1) + u(x-1,y) - u(x+1,y) - u(x-1,y+1))/(2*dxdy) # 2dx,dy
# uxy(x,y) = (u(x+1,y+1) + u(x,y-1) - u(x+1,y-1) - u(x,y+1))/(dx2*dy) # dx,2dy
# uxy(x,y) = (u(x+1,y+1) + u(x,y) - u(x+1,y) - u(x,y+1))/(dxdy) # dx,dy

num_dof = Nx.prod()
mat_xv = np.zeros((num_dof,num_dof)) # No choice but to use a dense matrix
for i in range(Nx[0]):
    for j in range(Nx[1]):
        pos = (i,j)
        ind = get_ind(Nx,pos)
        denom = 4.0*dx.prod()
        # Mind bounds
        if i==0:
            ind_i = (i,i+1)
            denom /= 2.0
        elif i==Nx[0]-1:
            ind_i = (i-1,i)
            denom /= 2.0
        else:
            ind_i = (i-1,i+1)
        if j==0:
            ind_j = (j,j+1)
            denom /= 2.0
        elif j==Nx[1]-1:
            ind_j = (j-1,j)
            denom /= 2.0
        else:
            ind_j = (j-1,j+1)

        ind_1 = get_ind(Nx,(ind_i[1],ind_j[1])) # x+1,y+1
        ind_2 = get_ind(Nx,(ind_i[0],ind_j[0])) # x-1,y-1
        ind_3 = get_ind(Nx,(ind_i[1],ind_j[0])) # x+1,y-1
        ind_4 = get_ind(Nx,(ind_i[0],ind_j[1])) # x-1,y+1
        coef_xy = rho*sigma*get_v(dx,pos)/denom
        mat_xv[ind,ind_1] = coef_xy
        mat_xv[ind,ind_2] = coef_xy # ind_2
        mat_xv[ind,ind_3] = -coef_xy # ind_2
        mat_xv[ind,ind_4] = -coef_xy # ind_2
        

# Second, construct Pxx + Px
mat_xx = np.zeros((num_dof,num_dof))
for i in range(Nx[0]):
    for j in range(Nx[1]):
        pos = (i,j)
        ind = get_ind(Nx,pos)
        # -0.5*r term
        mat_xx[ind,ind] += -0.5*r
        # fxx
        if (0<i<Nx[0]-1): # zero-gamma
            ind_m = get_ind(Nx,(i-1,j))
            ind_p = get_ind(Nx,(i+1,j))
            fxx = np.array([1,-2,1])/dx[0]**2
            coef_xx = 0.5*get_v(dx,pos)
            Pxx = fxx*coef_xx
            mat_xx[ind,ind_m] += Pxx[0]
            mat_xx[ind,ind] += Pxx[1]
            mat_xx[ind,ind_p] += Pxx[2]
        # fx
        coef_x = r - 0.5*sigma**2 *get_v(dx,pos)
        if (0<i<Nx[0]-1):
            ind_m = get_ind(Nx,(i-1,j))
            ind_p = get_ind(Nx,(i+1,j))
            fx = np.array([-1,1])/(2.0*dx[0])
            Px = fx*coef_x
            mat_xx[ind,ind_m] += Px[0]
            mat_xx[ind,ind_p] += Px[1]
        if i==0:
            ind_p = get_ind(Nx,(i+1,j))
            fx = np.array([-1,1])/dx[0]
            Px = fx*coef_x
            mat_xx[ind,ind] += Px[0]
            mat_xx[ind,ind_p] += Px[1]
        elif i==Nx[0]-1:
            ind_m = get_ind(Nx,(i-1,j))
            fx = np.array([-1,1])/dx[0]
            Px = fx*coef_x
            mat_xx[ind,ind_m] += Px[0]
            mat_xx[ind,ind] += Px[1]

# New space
mat_xx_new = np.zeros((num_dof,num_dof))
for i in range(Nx[0]):
    for j in range(Nx[1]):
        pos = (i,j)
        ind = get_ind(Nx,pos)
        ind = i + Nx[0]*j # new
        # -0.5*r term
        mat_xx_new[ind,ind] += -0.5*r
        # fxx
        if (0<i<Nx[0]-1): # zero-gamma
            ind_m = (i-1) + Nx[0]*j
            ind_p = (i+1) + Nx[0]*j
            fxx = np.array([1,-2,1])/dx[0]**2
            coef_xx = 0.5*get_v(dx,pos)
            Pxx = fxx*coef_xx
            mat_xx_new[ind,ind_m] += Pxx[0]
            mat_xx_new[ind,ind] += Pxx[1]
            mat_xx_new[ind,ind_p] += Pxx[2]
        # fx
        coef_x = r - 0.5*sigma**2 *get_v(dx,pos)
        if (0<i<Nx[0]-1):
            ind_m = (i-1) + Nx[0]*j
            ind_p = (i+1) + Nx[0]*j
            fx = np.array([-1,1])/(2.0*dx[0])
            Px = fx*coef_x
            mat_xx_new[ind,ind_m] += Px[0]
            mat_xx_new[ind,ind_p] += Px[1]
        if i==0:
            ind_p = (i+1) + Nx[0]*j
            fx = np.array([-1,1])/dx[0]
            Px = fx*coef_x
            mat_xx_new[ind,ind] += Px[0]
            mat_xx_new[ind,ind_p] += Px[1]
        elif i==Nx[0]-1:
            ind_m = (i-1) + Nx[0]*j
            fx = np.array([-1,1])/dx[0]
            Px = fx*coef_x
            mat_xx_new[ind,ind_m] += Px[0]
            mat_xx_new[ind,ind] += Px[1]
            

# Third, construct Pvv + Pv terms
mat_vv = np.zeros((num_dof,num_dof))
for i in range(Nx[0]):
    for j in range(Nx[1]):
        pos = (i,j)
        ind = get_ind(Nx,pos)
        # -0.5*r term
        mat_vv[ind,ind] += -0.5*r
        # fvv
        if (0<j<Nx[1]-1): # zero-gamma
            ind_m = get_ind(Nx,(i,j-1))
            ind_p = get_ind(Nx,(i,j+1))
            fvv = np.array([1,-2,1])/dx[1]**2
            coef_vv = 0.5*sigma**2*get_v(dx,pos)
            Pvv = fvv*coef_vv
            mat_vv[ind,ind_m] += Pvv[0]
            mat_vv[ind,ind] += Pvv[1]
            mat_vv[ind,ind_p] += Pvv[2]
        # fv
        coef_v = kappa*(theta - get_v(dx,pos))
        if (0<j<Nx[1]-1):
            ind_m = get_ind(Nx,(i,j-1))
            ind_p = get_ind(Nx,(i,j+1))
            fv = np.array([-1,1])/(2.0*dx[1])
            Pv = fv*coef_v
            mat_vv[ind,ind_m] += Pv[0]
            mat_vv[ind,ind_p] += Pv[1]
        if j==0:
            ind_p = get_ind(Nx,(i,j+1))
            fv = np.array([-1,1])/dx[1]
            Pv = fv*coef_v
            mat_vv[ind,ind] += Pv[0]
            mat_vv[ind,ind_p] += Pv[1]
        elif j==Nx[1]-1:
            ind_m = get_ind(Nx,(i,j-1))
            fv = np.array([-1,1])/dx[1]
            Pv = fv*coef_v
            mat_vv[ind,ind_m] += Pv[0]
            mat_vv[ind,ind] += Pv[1]
            
# Finally, think about how to convert Pxx+Px in [v,x] to [x,v] to fully utilize the efficiency of tridiagonal matrix

# Convert mat_xx to a tridiagonal matrix

In [12]:
# conventional (0,0),(0,1),(0,2),...,(1,0), ... , (Nx[0]-1,Nx[1]-1)
# I want (0,0),(1,0),(2,0),...,(0,1), ... , (Nx[0]-1,Nx[1]-1)
# Construct fwd and bwd converters to reshape a matrix to be a diagonal
fwd_convert = OrderedDict() # conventional -> new
for i in range(Nx[0]):
    for j in range(Nx[1]):
        pos = (i,j)
        i_c = get_ind(Nx,pos) # conventional
        i_n = i + Nx[0]*j # new
        fwd_convert[i_c] = i_n
bwd_convert = dict({y:x for x,y in fwd_convert.items()}) # switch key and item
bwd_convert = OrderedDict(sorted(bwd_convert.items(), key=lambda t: t[0])) # sort keys

# Solution Process Douglas Scheme

In [16]:
mat_vv

array([[  -1.55      ,    1.55      ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [  12.29166667,  -25.83333333,   13.54166667, ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,   25.35833333,  -51.66666667, ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ..., -749.16666667,
         371.00833333,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,  391.225     ,
        -775.        ,  383.775     ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           7.75      ,   -7.75      ]])

In [14]:
np.dot(mat_vv,P0)

array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
tr = tridiagonal(matrix=mat_vv)
tr*P0

array([0., 0., 0., ..., 0., 0., 0.])

In [22]:
tr.diag_3

array([  1.55      ,  13.54166667,  26.30833333, ..., 358.24166667,
       371.00833333, 383.775     ])

In [13]:
# Initial value for the call option
th_douglas = 1.0
P0 = np.zeros(num_dof)
for i in range(Nx[0]):
    for j in range(Nx[1]):
        pos = (i,j)
        ind = get_ind(Nx,pos)
        x,_ = get_x(dx,pos,x_min)
        S = np.exp(x)
        P0[ind] = np.maximum(S-k,0.0) # initial value (payoff at maturity)

Pn = copy.deepcopy(P0) # next
Pc = copy.deepcopy(P0) # current

# Douglas scheme            
# transformed matrices and vectors
print("Prepare transformed mat_1_new")
rhs_new = np.zeros(num_dof)
mat_1_new = np.eye(num_dof) - dt*th_douglas*mat_xx_new # time-consuming
tri_1_new = tridiagonal(matrix=mat_1_new)
# mat_1 = np.eye(num_dof) - dt*th_douglas*mat_xx
# mat_1_new = np.zeros(mat_1.shape)
# for i_c in range(num_dof):
#     for j_c in range(num_dof):
#         i_n = fwd_convert[i_c]; j_n = fwd_convert[j_c]
#         mat_1_new[i_n,j_n] = mat_1[i_c,j_c]
# tri_1_new = tridiagonal(matrix=mat_1_new)
print("Completed transformed tri_1_new")

mat_2 = np.eye(num_dof) - dt*th_douglas*mat_vv
tri_2 = tridiagonal(matrix=mat_2)
tri_vv = tridiagonal(matrix=mat_vv)
        
Y1 = np.zeros(num_dof)
for i,t in enumerate(t_space):
    if i==0: continue # skip the very first
    Pc[:] = Pn[:]
    # Explicit step
    Y0 = Pc + dt*(np.dot(mat_xv,Pc) + np.dot(mat_xx,Pc) + np.dot(mat_vv,Pc)) # np.dot is better
    # First implicit step
    tmp = Y0 - dt*th_douglas*np.dot(mat_xx,Pc)
    for ii in range(num_dof): # Convert the vector into a new space
        rhs_new[fwd_convert[ii]] = tmp[ii]
    Y1_tmp = tri_1_new.solve(rhs_new) # solution in a new space
    # Transform the solution back to the convention
    for ii in range(num_dof):
        Y1[bwd_convert[ii]] = Y1_tmp[ii] # convert to the conventional space
    print(Y0)
    print(Y1)
    Y2 = tri_2.solve(Y1 - dt*th_douglas*(tri_vv*Pc))
    print(Y2)
    Pn[:] = Y2[:]
Pc[:] = Pn[:]


Prepare transformed mat_1_new
Completed transformed tri_1_new
[0.         0.         0.         ... 3.4692999  3.46887269 3.46844548]
[ 0.00000000e+00 -2.82816492e-31 -3.08120965e-26 ...  3.47003811e+00
  3.46965447e+00  3.46927174e+00]
[-7.54445003e-16 -6.59530954e-15 -1.77414763e-14 ...  3.47027189e+00
  3.46989379e+00  3.46951584e+00]
[-1.50889001e-15 -8.73415060e-14 -2.97664977e-13 ...  3.46341754e+00
  3.46280155e+00  3.46219719e+00]
[-1.50889001e-15 -8.75481757e-14 -2.98778019e-13 ...  3.46388959e+00
  3.46329864e+00  3.46271953e+00]
[-1.10046600e-13 -9.56179412e-13 -2.49979303e-12 ...  3.46406773e+00
  3.46346529e+00  3.46286386e+00]
[-2.19338755e-13 -2.43458099e-12 -6.76510702e-12 ...  3.46023250e+00
  3.45947050e+00  3.45872166e+00]
[-2.19338755e-13 -2.43620956e-12 -6.77348443e-12 ...  3.46057176e+00
  3.45982710e+00  3.45909571e+00]
[-1.01506340e-12 -8.02164507e-12 -2.01066059e-11 ...  3.46070889e+00
  3.45995122e+00  3.45919541e+00]
[-1.92008020e-12 -1.71675044e-11 -4.441361

In [6]:
rhs_new = np.zeros(num_dof)

mat_1_new = np.eye(num_dof) - dt*th_douglas*mat_xx_new


In [7]:
mat_1_new = copy.deepcopy(mat_xx_new)

In [8]:
mat_xx_new.nbytes/ (2**(10*3)) # we are not going to use it

0.0078125

In [9]:
mat_xx_new.nbytes

8388608

In [11]:
x_ind = get_x_ind(log_k,dx[0],x_min[0])
v_ind = get_x_ind(v0,dx[1],x_min[1])
ind = get_ind(Nx,(x_ind,v_ind))
ind_p = get_ind(Nx,(x_ind+1,v_ind+1))
x = get_x(dx,(x_ind,v_ind),x_min)
x_p = get_x(dx,(x_ind+1,v_ind+1),x_min)
u = (log_k-x_p[0])/(x[0]-x_p[0])
P_interpolated = u*Pc[ind]+(1-u)*Pc[ind_p]
sol3 = P_interpolated
print('Interpolated ',P_interpolated) # reference 0.06542118636775274

Interpolated  0.05915433079170907
